In [16]:
import sys
sys.path.append('C:\\Users\\gabri\\my_projects\\stock_analysis')

import plotly.express as px
import pandas as pd
from stock import yfScraper, StockAnalyzer
import numpy as np
import plotly.graph_objects as go
import textwrap

In [2]:
scrape = yfScraper(r'C:\Users\gabri\my_projects\stock_analysis\ticker_attributes.json')
df = scrape.attributes_df()
df = df[df['ticker_info.security'].notna()]

File exists, successfully loaded attributes file: C:\Users\gabri\my_projects\stock_analysis\ticker_attributes.json


In [3]:
df['ticker_summary.avg_volume'] = df['ticker_summary.avg_volume'].astype(str)
df['ticker_summary.market_cap'] = df['ticker_summary.market_cap'].astype(str)
df['ticker_summary.pe_ratio'] = df['ticker_summary.pe_ratio'].astype(str)
df['ticker_summary.eps'] = df['ticker_summary.eps'].astype(str)
df['ticker_summary.beta'] = df['ticker_summary.beta'].astype(str)

df['ticker_summary.avg_volume'] = pd.to_numeric(df['ticker_summary.avg_volume'].str.replace(',', ''), errors='coerce')
df['ticker_summary.market_cap'] = df['ticker_summary.market_cap'].replace({'B': 'e9', 'T': 'e12'}, regex=True)
df['ticker_summary.market_cap'] = pd.to_numeric(df['ticker_summary.market_cap'], errors='coerce')
df['ticker_summary.pe_ratio'] = pd.to_numeric(df['ticker_summary.pe_ratio'], errors='coerce')
df['ticker_summary.eps'] = pd.to_numeric(df['ticker_summary.eps'], errors='coerce')
df['ticker_summary.beta'] = pd.to_numeric(df['ticker_summary.beta'], errors='coerce')


# df['ticker_summary.date'] = pd.to_datetime(df['ticker_summary.date'])
# df['ticker_info.date_added'] = pd.to_datetime(df['ticker_info.date_added'])
# df['ticker_summary.regular_market_price'] = pd.to_numeric(df['ticker_summary.regular_market_price'], errors='coerce')
# df['ticker_summary.regular_market_change'] = pd.to_numeric(df['ticker_summary.regular_market_change'], errors='coerce')
# # df['ticker_summary.regular_market_change_percent'] = df['ticker_summary.regular_market_change_percent'].str.replace('[()%]', '').astype(float) / 100
# df['ticker_summary.post_market_price'] = pd.to_numeric(df['ticker_summary.post_market_price'], errors='coerce')
# df['ticker_summary.post_market_change'] = pd.to_numeric(df['ticker_summary.post_market_change'], errors='coerce')
# # df['ticker_summary.post_market_change_percent'] = df['ticker_summary.post_market_change_percent'].str.replace('[()%]', '').astype(float) / 100
# df['ticker_summary.previous_close'] = pd.to_numeric(df['ticker_summary.previous_close'], errors='coerce')
# df['ticker_summary.open_value'] = pd.to_numeric(df['ticker_summary.open_value'], errors='coerce')
# df['ticker_summary.bid'] = pd.to_numeric(df['ticker_summary.bid'].str.split(' x ').str[0], errors='coerce')
# df['ticker_summary.ask'] = pd.to_numeric(df['ticker_summary.ask'].str.split(' x ').str[0], errors='coerce')
# df['ticker_summary.volume'] = pd.to_numeric(df['ticker_summary.volume'].str.replace(',', ''), errors='coerce')
#     df['ticker_summary.avg_volume'] = pd.to_numeric(df['ticker_summary.avg_volume'].str.replace(',', ''), errors='coerce')
#     df['ticker_summary.market_cap'] = df['ticker_summary.market_cap'].replace({'B': 'e9', 'T': 'e12'}, regex=True).astype(float)

# # df['ticker_summary.beta'] = pd.to_numeric(df['ticker_summary.beta'], errors='coerce')
#     df['ticker_summary.pe_ratio'] = pd.to_numeric(df['ticker_summary.pe_ratio'], errors='coerce')
#     df['ticker_summary.eps'] = pd.to_numeric(df['ticker_summary.eps'], errors='coerce')
# # df['ticker_summary.earnings_date'] = df['ticker_summary.earnings_date'].str.split(' - ').apply(lambda x: pd.to_datetime(x[0]) if isinstance(x, list) else pd.NaT)
# df['ticker_summary.forward_dividend_yield'] = df['ticker_summary.forward_dividend_yield'].str.split(' ').str[0].astype(float)
# df['ticker_summary.ex_dividend_date'] = pd.to_datetime(df['ticker_summary.ex_dividend_date'])
# df['ticker_summary.year_target_est'] = pd.to_numeric(df['ticker_summary.year_target_est'], errors='coerce')
# df['ticker_stock_price.from_unix'] = pd.to_numeric(df['ticker_stock_price.from_unix'], errors='coerce')
# df['ticker_stock_price.to_unix'] = pd.to_numeric(df['ticker_stock_price.to_unix'], errors='coerce')
# df['ticker_stock_price.from_date'] = pd.to_datetime(df['ticker_stock_price.from_date'])
# df['ticker_stock_price.to_date'] = pd.to_datetime(df['ticker_stock_price.to_date'])


In [4]:
sectors, counts = np.unique(df[df['ticker_info.gics_sector'].notna()]['ticker_info.gics_sector'], return_counts=True)
sector_sub_industries = df[['ticker_info.gics_sector', 'ticker_info.gics_sub_industry']].drop_duplicates()
sector_to_sub_industry = sector_sub_industries.groupby('ticker_info.gics_sector')['ticker_info.gics_sub_industry'].apply(lambda x: ', '.join(x)).to_dict()
def wrap_text(text, width=40):
    return '<br>'.join(textwrap.wrap(text, width=width))
hover_text = [wrap_text(sector_to_sub_industry[sector]) for sector in sectors]
fig = go.Figure(go.Treemap(
    labels=sectors,
    parents=[''] * len(sectors),  # No parents since all are top-level
    values=counts,
    hovertext=hover_text,
    hoverinfo='label+text+value+percent entry'
))
fig.update_layout(title_text='Sector Distribution', width=1000, height=400)
fig.show()

In [5]:
filtered_df = df[df['ticker_info.gics_sector'].notna()][['ticker_info.gics_sub_industry', 'ticker_info.security']]
grouped_df = filtered_df.groupby('ticker_info.gics_sub_industry').agg({'ticker_info.security': lambda x: ', '.join(x)}).reset_index()
treemap_data = filtered_df.groupby(['ticker_info.gics_sub_industry']).size().reset_index(name='Counts')
treemap_data = pd.merge(treemap_data, grouped_df, on='ticker_info.gics_sub_industry', how='left')

fig = px.treemap(treemap_data, path=['ticker_info.gics_sub_industry'], values='Counts', hover_data='ticker_info.security', title='Sub-Industry Distribution')
fig.show()

In [6]:
filtered_df = df[df['ticker_info.gics_sub_industry'].notna()][['ticker_info.founded', 'ticker_info.security', 'ticker_info.gics_sector']]
filtered_df = filtered_df.sort_values(by='ticker_info.founded')
filtered_df['ticker_info.founded'] = filtered_df['ticker_info.founded'].astype(str)
fig = px.bar(filtered_df, x='ticker_info.founded', hover_data=['ticker_info.security'], color='ticker_info.gics_sector')
fig.update_xaxes(categoryorder='category ascending')
fig.show()

In [7]:
df['ticker_info.date_added'] = pd.to_datetime(df['ticker_info.date_added'])
df['year_added'] = df['ticker_info.date_added'].dt.year

filtered_df = df[df['ticker_info.gics_sub_industry'].notna()][['year_added', 'ticker_info.security', 'ticker_info.gics_sector']]
filtered_df = filtered_df.sort_values(by='year_added')
filtered_df['year_added'] = filtered_df['year_added'].astype(str)
fig = px.bar(filtered_df, x='year_added', hover_data=['ticker_info.security'], color='ticker_info.gics_sector')
fig.update_xaxes(categoryorder='category ascending')
fig.show()

In [8]:
filtered_df = df[df['ticker_info.gics_sub_industry'].notna()][['ticker_info.founded', 'ticker_info.security', 'year_added', 'ticker_info.gics_sector']]
filtered_df = filtered_df.sort_values(by='ticker_info.founded')
fig = px.scatter(filtered_df, x='year_added', y='ticker_info.founded', hover_data=['ticker_info.security'], color='ticker_info.gics_sector')
fig.show()

In [9]:
selected_columns = ['ticker_info.security', 'ticker_summary.pe_ratio', 'ticker_summary.market_cap', 'ticker_info.gics_sub_industry', 'ticker_info.gics_sector']
df1 = df[selected_columns].sort_values('ticker_summary.pe_ratio')
fig = px.box(df1, x='ticker_info.gics_sector', y='ticker_summary.pe_ratio',
             title='Boxplot of PE Ratio by GICS Sector',
             labels={'ticker_info.gics_sector': 'GICS Sector', 'ticker_summary.pe_ratio': 'PE Ratio'})
fig.update_layout(
    xaxis_title='GICS Sector',
    yaxis_title='PE Ratio',
    title='Boxplot of PE Ratio by GICS Sector'
)
fig.show()

In [10]:
selected_columns = ['ticker_info.security', 'ticker_summary.pe_ratio', 'ticker_summary.market_cap', 'ticker_summary.beta']
df1 = df[df['ticker_info.gics_sub_industry'] == 'Aerospace & Defense'][selected_columns].sort_values('ticker_summary.pe_ratio')
df1 = df[selected_columns].sort_values('ticker_summary.pe_ratio')

df1 = df1.dropna()
fig = px.scatter(data_frame=df1,
           x='ticker_info.security',
           y='ticker_summary.pe_ratio',
           size='ticker_summary.market_cap',
           color ='ticker_summary.beta')
fig.update_layout(width=800, height=500)
fig.show()

In [11]:
df1 = df.copy()
selected_columns = ['ticker_info.security', 'ticker_summary.pe_ratio', 'ticker_summary.market_cap', 'ticker_summary.beta']

# gics_sub_industry = 'Technology Hardware, Storage & Peripherals' #'Semiconductors'
# df1 = df1[df1['ticker_info.gics_sub_industry'] == gics_sub_industry][selected_columns].sort_values('ticker_summary.pe_ratio')

gics_sector = 'Information Technology'	
df1 = df1[df1['ticker_info.gics_sector'] == gics_sector][selected_columns].sort_values('ticker_summary.pe_ratio')

df1['ticker_summary.market_cap_billions'] = df1['ticker_summary.market_cap'] / 1e9
df1 = df1.dropna()
fig = px.scatter(data_frame=df1,
           x='ticker_info.security',
           y='ticker_summary.pe_ratio',
           size='ticker_summary.market_cap_billions',
           color='ticker_summary.beta')
fig.update_traces(marker=dict(sizeref=0.5*max(df1['ticker_summary.market_cap_billions'])/(40.**2), sizemode='area'))

fig.update_layout(width=1800, height=500, plot_bgcolor='white', xaxis=dict(tickangle=45))
fig.show()

In [27]:
stock_analysis = StockAnalyzer(scrape)
stock_analysis.line_plot(['MSFT'])

KeyError: 'processed_csv_destination'

In [12]:
df_price_hist = scrape.get_stock_price_dfs(['^GSPC'])
df_price_hist

,date,open,high,low,close,adj_close,volume,ticker
0,2024-05-24,5281.45,5311.65,5278.39,5304.72,5304.72,3.005510e+09,^GSPC
1,2024-05-23,5340.26,5341.88,5256.93,5267.84,5267.84,3.869520e+09,^GSPC
2,2024-05-22,5319.28,5323.18,5286.01,5307.01,5307.01,3.847130e+09,^GSPC
3,2024-05-21,5298.69,5324.32,5297.87,5321.41,5321.41,3.662240e+09,^GSPC
4,2024-05-20,5305.35,5325.32,5302.40,5308.13,5308.13,3.420100e+09,^GSPC
...,...,...,...,...,...,...,...,...
24211,1928-01-06,17.66,17.66,17.66,17.66,17.66,NaN,^GSPC
24212,1928-01-05,17.55,17.55,17.55,17.55,17.55,NaN,^GSPC
24213,1928-01-04,17.72,17.72,17.72,17.72,17.72,NaN,^GSPC
24214,1928-01-03,17.76,17.76,17.76,17.76,17.76,NaN,^GSPC


In [13]:
df_price_hist

,date,open,high,low,close,adj_close,volume,ticker
0,2024-05-24,5281.45,5311.65,5278.39,5304.72,5304.72,3.005510e+09,^GSPC
1,2024-05-23,5340.26,5341.88,5256.93,5267.84,5267.84,3.869520e+09,^GSPC
2,2024-05-22,5319.28,5323.18,5286.01,5307.01,5307.01,3.847130e+09,^GSPC
3,2024-05-21,5298.69,5324.32,5297.87,5321.41,5321.41,3.662240e+09,^GSPC
4,2024-05-20,5305.35,5325.32,5302.40,5308.13,5308.13,3.420100e+09,^GSPC
...,...,...,...,...,...,...,...,...
24211,1928-01-06,17.66,17.66,17.66,17.66,17.66,NaN,^GSPC
24212,1928-01-05,17.55,17.55,17.55,17.55,17.55,NaN,^GSPC
24213,1928-01-04,17.72,17.72,17.72,17.72,17.72,NaN,^GSPC
24214,1928-01-03,17.76,17.76,17.76,17.76,17.76,NaN,^GSPC


In [14]:
px.line(data_frame=df_price_hist, x='date', y='close', color='ticker')